In [173]:
from skl2onnx import to_onnx
from sklearn.ensemble import RandomForestClassifier
import sqlalchemy as db
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from skl2onnx.common.data_types import FloatTensorType, Int16TensorType, DoubleTensorType
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report  

from mclLib.featurenames import *
from mclLib.modelpostfix import *
from mclLib.cleaner import *
from mclLib.server_info import *
from mclLib.tester import *

In [174]:

engine = create_engine(mysql_server_uri)
conn = engine.connect()


Exception during reset or similar
Traceback (most recent call last):
  File "c:\Users\MJ\anaconda3\envs\py310\lib\site-packages\sqlalchemy\pool\base.py", line 670, in _finalize_fairy
    fairy._reset(pool)
  File "c:\Users\MJ\anaconda3\envs\py310\lib\site-packages\sqlalchemy\pool\base.py", line 875, in _reset
    pool._dialect.do_rollback(self)
  File "c:\Users\MJ\anaconda3\envs\py310\lib\site-packages\sqlalchemy\engine\default.py", line 642, in do_rollback
    dbapi_connection.rollback()
MySQLdb.OperationalError: (2006, 'Server has gone away')


In [175]:
''' get db data '''
br_full_data = pd.read_sql_table('buyreports', conn)

In [179]:
''' db data filter '''
get_filter = ( br_full_data['isAllBuyed'] == 1) & ( br_full_data['isAllSelled'] == 1) & (br_full_data['dTradeTime'] >= '2023-03-01') 
br = br_full_data[get_filter]

''' get features name'''
feature_names =  f_name_102
feature_size = len(feature_names)

model_name = 'RF_pow30_06_n250_md10'

''' set X data '''
X = br[feature_names].to_numpy(dtype=np.float64)

''' set y data '''
y_condition = ((br['fMaxPowerAfterBuyWhile30'] <= 0.04)) # | ((br['lTotalBuyEndPrice'] - br['lTotalBuyPrice']) <= 1000000000))
y = np.where(y_condition, 1, 0)

In [177]:
from collections import Counter
Counter(y)

Counter({1: 100061, 0: 28284})

In [160]:
# smote = SMOTE(sampling_strategy=0.85)
# X, y = smote.fit_resample(X, y)

In [178]:
rf = RandomForestClassifier(n_estimators=200, max_depth=14)
rf.fit(X, y)

RandomForestClassifier(max_depth=14, n_estimators=200)

In [156]:
onx = to_onnx(model=rf, options={'zipmap': False},
              initial_types=[('input', DoubleTensorType([None, 102]))],
              final_types=[('output', DoubleTensorType([None])),
                           ('prob', DoubleTensorType([None, 2]))],
              target_opset=17)

with open(onnx_path + model_name + onnx_, "wb") as f:
    f.write(onx.SerializeToString())

In [157]:
cleanAll()

In [180]:
y_pred = rf.predict(X)

In [181]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.48      0.08      0.14     15358
           1       0.82      0.98      0.89     66819

    accuracy                           0.81     82177
   macro avg       0.65      0.53      0.52     82177
weighted avg       0.76      0.81      0.75     82177



In [182]:
testClassification(y, [y_pred])

총량 :  82177
0 :  15358 , 비율 :  18.688927558805993 (%)
1 :  66819 , 비율 :  81.31107244119401 (%)

============ predict 0 =============
총 횟수 :  2540 ,  타겟기준 :  0.0
실제 0 :  1221
실제 1 :  1319
정답비율 :  48.07086614173228 (%)

============ predict 1 =============
총 횟수 :  79637 , 타겟기준 :  1.0
실제 1 :  65500
실제 0 :  14137
정답비율 :  82.248201213004 (%)

